In [101]:
# This notebook does the following:
    # A. Queries our database to construct sentence level data from court commitment and sentence computation for every
    # infraction resulting in incarceration. (dataset A)
    # B. Queries sentence component to get Most Serious Offense from all sentence components since this variable
    # is missing in much of dataset A and is needed as our outcome variable (dataset B)
    # C. Puts together dataset A and B
    # D. Carries out several steps of cleaning the data and getting recidivism flag
    # E. Queries database for any additional features (e.g. disciplinary infractions)
    # F. Hold outs active sentences, drops those missing recidivism flag
    # Dropped observations missing the following (if we can't proxy for them)
        # Sentence Start Date (~1.3%)
        # Sentence End Date (~800 obs)
        # Most Serious Offense (2.6% obs)
        # Our decided category (~1% obs)
    # F. 

In [102]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import config
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd

import importlib

import datetime
import re
import numpy as np

In [103]:
importlib.reload(qd)

<module 'query_db' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/query_db.py'>

In [104]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [105]:
#coded_offenses = pd.read_excel('https://github.com/christi-liongson/covid_decarceration/blob/construct_public_safety_data/data/Coding%20Offenses%20-%20For%20GitHub.xlsx',sheet_name="Coding - FINAL")
coded_offenses = pd.read_excel('../data/Coding Offenses - For GitHub.xlsx',sheet_name="Coding - FINAL")

In [106]:
coded_offenses.head()

,Primary offense code,Description (if needed),Decided Category,Needed a check?,More lenient,More harsh
0,DRIV LICENSE REVOKED,0,1,NO,1,1
1,LARCENY,0,2,YES,1,3
2,DWI DRIVING WHILE IMPAIRED,0,2,NO,2,2
3,FELONY B&E,"Felony Breaking and Entering, as opposed to Mi...",3,YES,2,4
4,WORTHLESS CHECK,0,1,NO,1,1


In [107]:
# Part A: Queries our database to construct sentence level data from court commitment and sentence computation for every
    # infraction resulting in incarceration. (dataset A)
start = datetime.datetime.now()
query_court_commitment = '''
                        SELECT A.OFFENDER_NC_DOC_ID_NUMBER as ID, 
                            A.COMMITMENT_PREFIX, 
                            A.EARLIEST_SENTENCE_EFFECTIVE_DT, 
                            A.MOST_SERIOUS_OFFENSE_CODE                              
                        FROM OFNT3BB1 A
                        WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y";
                        '''

conn = create_connection(config.database_name)
court_small = qd.query_db_notebook(conn,query_court_commitment)


query_sentence_comp = '''
                            SELECT INMATE_DOC_NUMBER as ID, 
                                INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, 
                                INMATE_COMPUTATION_STATUS_FLAG, 
                                max(ACTUAL_SENTENCE_END_DATE) as END_DATE,
                                max(PROJECTED_RELEASE_DATE_PRD) as PROJ_END_DATE
                            FROM INMT4BB1
                            GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX;
                        '''

sentence_compute_small = qd.query_db_notebook(conn,query_sentence_comp)


query_inmt_profile = '''
                    SELECT 
                        INMATE_DOC_NUMBER as ID,
                        INMATE_RECORD_STATUS_CODE,
                        INMATE_ADMIN_STATUS_CODE,
                        DATE_OF_LAST_INMATE_MOVEMENT,
                        TYPE_OF_LAST_INMATE_MOVEMENT,
                        CURRENT_COMMITMENT_PREFIX,
                        INMATE_GENDER_CODE as GENDER,
                        INMATE_RACE_CODE as RACE,
                        INMATE_BIRTH_DATE as BIRTH_DATE,
                        INMATE_ETHNIC_AFFILIATION as ETHNICITY,
                        INMATE_CONTROL_STATUS_CODE as CONTROL_STATUS,
                        INMATE_SPECIAL_CHARACTERISTICS as SPECIAL_CHARS,
                        TOTAL_DISCIPLINE_INFRACTIONS,
                        LATEST_DISCIPLINE_INFRACTION,
                        LAST_DISCIPLINE_INFRACTION_DT
                    FROM INMT4AA1;
                    '''

query_inmt_profile = '''
                    SELECT 
                        INMATE_DOC_NUMBER as ID,
                        INMATE_RECORD_STATUS_CODE,
                        INMATE_ADMIN_STATUS_CODE,
                        DATE_OF_LAST_INMATE_MOVEMENT,
                        TYPE_OF_LAST_INMATE_MOVEMENT,
                        CURRENT_COMMITMENT_PREFIX,
                        INMATE_CONTROL_STATUS_CODE as CONTROL_STATUS
                    FROM INMT4AA1;
                    '''

inmt_profile = qd.query_db_notebook(conn,query_inmt_profile)

query_offender_profile = '''
                        SELECT 
                        OFFENDER_NC_DOC_ID_NUMBER as ID,
                        OFFENDER_GENDER_CODE as GENDER,
                        OFFENDER_RACE_CODE as RACE,
                        OFFENDER_BIRTH_DATE as BIRTH_DATE,
                        STATE_WHERE_OFFENDER_BORN as STATE_BORN,
                        OFFENDER_ETHNIC_CODE as ETHNICITY,
                        OFFENDER_CITIZENSHIP_CODE as CITIZENSHIP                        
                    FROM OFNT3AA1;
                            
                        '''

offender_profile = qd.query_db_notebook(conn,query_offender_profile)

conn.close

data = court_small.merge(sentence_compute_small, on=['ID','COMMITMENT_PREFIX'], how='outer')
data = data.merge(inmt_profile, on=['ID'], how = 'left')
data = data.merge(offender_profile, on=['ID'], how = 'left')
#data = data.merge(disc_infraction, on=['ID'], how='left')


stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


Time Elapsed: 0:03:31.354339


In [108]:
#data.to_csv('datasetA_court_sentcomp.csv', index=False)

In [109]:
data.shape

(903181, 19)

In [110]:
# Part B: Queries sentence component to get Most Serious Offense from all sentence components since this variable
    # is missing in much of dataset A and is needed as our outcome variable (dataset B)

start = datetime.datetime.now()

query_sentence_component = '''
                            SELECT OFFENDER_NC_DOC_ID_NUMBER as ID, 
                                        COMMITMENT_PREFIX, 
                                        SENTENCE_COMPONENT_NUMBER,
                                        PRIMARY_OFFENSE_CODE,
                                        PRIMARY_FELONYMISDEMEANOR_CD,
                                        SENTENCING_PENALTY_CLASS_CODE,
                                        PRIOR_RECORD_LEVEL_CODE,
                                        MINIMUM_SENTENCE_LENGTH,
                                        MAXIMUM_SENTENCE_LENGTH,
                                        SENTENCE_TYPE_CODE,
                                        COUNTY_OF_CONVICTION_CODE
                            FROM OFNT3CE1
                            WHERE SENTENCE_TYPE_CODE LIKE '%PRISONS%';
                            '''

conn = create_connection(config.database_name)
sent_comp_small = qd.query_db_notebook(conn,query_sentence_component)

stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


Time Elapsed: 0:06:31.817429


In [111]:
#sent_comp_small.to_csv('datasetB_sentcomponent_only_incarcerated.csv', index=False)

In [112]:
# Part B: Queries sentence component to get Most Serious Offense from all sentence components since this variable
    # is missing in much of dataset A and is needed as our outcome variable (dataset B)
# Check how many unique ID and COMMITMENT_PREFIX combinations there are
dataset_B = sent_comp_small.copy()
grouped = dataset_B.groupby(['ID', 'COMMITMENT_PREFIX'])
total_combinations = grouped.ngroups
print(total_combinations)

# Find the ID / COMMITMENT_PREFIX combinations that have the maximum MINIMUM_SENTENCE_LENGTH
# We will use these combinations to filter dataset_B for PRIMARY_OFFENSE_CODE
# Note: These might not be unique

min_sentence = pd.DataFrame(dataset_B.groupby(['ID', 'COMMITMENT_PREFIX'])['MINIMUM_SENTENCE_LENGTH'].max().reset_index(name='max_min'))
min_sentence.head(10)

# Check to make sure we're not accidentally dropping any rows
min_sentence.groupby(['ID', 'COMMITMENT_PREFIX']).ngroups

# Filter dataset_B to only these rows
filter_tuples = [tuple(x) for x in min_sentence.to_numpy()]

filtered_B = dataset_B[dataset_B[['ID', 'COMMITMENT_PREFIX', 'MINIMUM_SENTENCE_LENGTH']].apply(tuple, axis=1).isin(filter_tuples)]
filtered_B.head(10)

count_nunique_offenses = pd.DataFrame(filtered_B.groupby(['ID', 'COMMITMENT_PREFIX'])['PRIMARY_OFFENSE_CODE'].nunique().reset_index(name='count'))
count_nunique_offenses['count'].describe()


# Pull out the ID / COMMITMENT_PREFIX combinations that are unique on max(MINIMUM_SENTENCE_LENGTH)
unique_min_filter = [tuple(x) for x in count_nunique_offenses[count_nunique_offenses['count'] == 1][['ID', 'COMMITMENT_PREFIX']].to_numpy()]
nonunique_min_filter = [tuple(x) for x in count_nunique_offenses[count_nunique_offenses['count'] != 1][['ID', 'COMMITMENT_PREFIX']].to_numpy()]

cols_to_keep = ['ID', 'COMMITMENT_PREFIX','PRIMARY_OFFENSE_CODE','MINIMUM_SENTENCE_LENGTH', 'MAXIMUM_SENTENCE_LENGTH']

filtered_B_min_unique = filtered_B[filtered_B[['ID','COMMITMENT_PREFIX']].apply(tuple, axis=1).isin(unique_min_filter)][cols_to_keep]
filtered_B_min_unique.head()

# Drop duplicate rows from filtered_B_min_unique (we know that they all have the same PRIMARY_OFFENSE_CODE)
# Note: This method keeps the first observation, but again, this shouldn't matter
filtered_B_min_unique.drop_duplicates(subset=['ID','COMMITMENT_PREFIX','PRIMARY_OFFENSE_CODE'],inplace=True)
filtered_B_min_unique.head()

filtered_B_min_nonunique = filtered_B[filtered_B[['ID','COMMITMENT_PREFIX']].apply(tuple, axis=1).isin(nonunique_min_filter)][cols_to_keep]
filtered_B_min_nonunique.head()

find_max_max = pd.DataFrame(filtered_B_min_nonunique.groupby(['ID', 'COMMITMENT_PREFIX'])['MAXIMUM_SENTENCE_LENGTH'].max().reset_index(name='max_max'))
find_max_max.head()

by_max_tuples = [tuple(x) for x in find_max_max.to_numpy()]
filtered_B_max = filtered_B_min_nonunique[filtered_B_min_nonunique[['ID', 'COMMITMENT_PREFIX', 'MAXIMUM_SENTENCE_LENGTH']].apply(tuple, axis=1).isin(by_max_tuples)]
filtered_B_max.head()

count_offenses_by_max = pd.DataFrame(filtered_B_max.groupby(['ID', 'COMMITMENT_PREFIX'])['PRIMARY_OFFENSE_CODE'].nunique().reset_index(name='count'))
count_offenses_by_max.head()

# Pull out the ID and COMMITMENT_PREFIX tuples in FILTERED_B_MT1 where there is a unique PRIMARY_OFFENSE_CODE
# after looking at the maximum of MAXIMUM_SENTENCE_LENGTH
unique_max = count_offenses_by_max[count_offenses_by_max['count'] == 1][['ID', 'COMMITMENT_PREFIX']]
unique_max_filter = [tuple(x) for x in unique_max.to_numpy()]

filtered_B_max_unique = filtered_B_max[filtered_B_max[['ID', 'COMMITMENT_PREFIX']].apply(tuple, axis=1).isin(unique_max_filter)]
filtered_B_max_unique.head()

# Drop duplicate rows from filtered_B_max_unique (we know that they all have the same PRIMARY_OFFENSE_CODE)
# Note: This method keeps the first observation, but again, this shouldn't matter
filtered_B_max_unique.drop_duplicates(subset=['ID','COMMITMENT_PREFIX','PRIMARY_OFFENSE_CODE'],inplace=True)
filtered_B_max_unique.head()

concat_1_2 = filtered_B_min_unique.append(filtered_B_max_unique)
concat_1_2.shape

# Final merged version of datasets A and B
dataset_with_most_serious = concat_1_2
dataset_with_most_serious.shape

891122


/Users/daminisharma/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(823722, 5)

In [113]:
# Part C: Puts together dataset A and B
datasetB_primary_offense = dataset_with_most_serious.loc[:,['ID','COMMITMENT_PREFIX','PRIMARY_OFFENSE_CODE']]

print("Dataset B # observations:",datasetB_primary_offense.shape[0])

# merging on datasetA (court commitment + sentence computation) with datasetB ("self constructed" primary offenses from
# sentence component)
data_A_B = data.merge(datasetB_primary_offense, on = ['ID','COMMITMENT_PREFIX'], how='left') 


Dataset B # observations: 823722


In [225]:
# Part D: Carries out several steps of cleaning the data and getting recidivism flag

# Replace Most Serious Offense with our constructed Primary Offense Code if missing
data_A_B['MOST_SERIOUS_OFFENSE_CODE'].mask(data_A_B['MOST_SERIOUS_OFFENSE_CODE'].isnull(), data_A_B['PRIMARY_OFFENSE_CODE'], inplace=True)

print("% missing most serious offense:",data_A_B['MOST_SERIOUS_OFFENSE_CODE'].isnull().sum() / data_A_B.shape[0])
print("Total number of observations in dataset A + B: ", data_A_B.shape[0])

# Step 1
# https://kanoki.org/2019/07/17/pandas-how-to-replace-values-based-on-conditions/
print("Cleaning dates and dropping missing")
data_A_B['END_DATE'].mask(data_A_B['END_DATE'] == '0001-01-01', data_A_B['PROJ_END_DATE'], inplace=True)
data_A_B = data_A_B[data_A_B['END_DATE']!='0001-01-01']
data_A_B = data_A_B[data_A_B['EARLIEST_SENTENCE_EFFECTIVE_DT']!='0001-01-01']
data_A_B = data_A_B[data_A_B['END_DATE'].notna()]
data_A_B = data_A_B[data_A_B['EARLIEST_SENTENCE_EFFECTIVE_DT'].notna()]

print("Total number of observations in dataset A + B: ", data_A_B.shape[0])
print("% still missing most serious offense:",data_A_B['MOST_SERIOUS_OFFENSE_CODE'].isnull().sum() / data_A_B.shape[0])

# Step 1.5 drop observations missing most serious offense code
print("Drop observations missing most serious offense code")
data_A_B = data_A_B[data_A_B['MOST_SERIOUS_OFFENSE_CODE'].notna()]
print("Total number of observations in dataset A + B: ", data_A_B.shape[0])

# Step 2
# write data to sqlite in memory so can query it to get next record
print("Querying database to get nextPrefix, nextOffense")
conn = sqlite3.connect(':memory:')
data_A_B.to_sql('data', conn, index=False)

start = datetime.datetime.now()
# https://stackoverflow.com/questions/37360901/sql-self-join-compare-current-record-with-the-record-of-the-previous-date
query_datasetAB = '''
                        SELECT *, 
                        LEAD(COMMITMENT_PREFIX,1,NONE) OVER (
                                                    PARTITION BY ID
                                                    ORDER BY COMMITMENT_PREFIX
                                                    ) NextPrefix,
                        LEAD(EARLIEST_SENTENCE_EFFECTIVE_DT,1,NONE) OVER (
                                                    PARTITION BY ID
                                                    ORDER BY COMMITMENT_PREFIX
                                                    ) NextStart,
                        LEAD(MOST_SERIOUS_OFFENSE_CODE,1,NONE) OVER (
                                                    PARTITION BY ID
                                                    ORDER BY COMMITMENT_PREFIX
                                                    ) NextOffense                                                    
                                                    
                        FROM data ;

                        '''


dataset_flag = qd.query_db_notebook(conn,query_datasetAB)
conn.close
stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


% missing most serious offense: 0.0
Total number of observations in dataset A + B:  864817
Cleaning dates and dropping missing
Total number of observations in dataset A + B:  864817
% still missing most serious offense: 0.0
Drop observations missing most serious offense code
Total number of observations in dataset A + B:  864817
Querying database to get nextPrefix, nextOffense
Time Elapsed: 0:02:42.930559


In [226]:
# Define functions that fix dates
# specifically, some dates are top coded as 9999- usually for a life sentence
# this exceeds pandas' max date, so they first need to be re-top-coded, then turned into the date format
# date == 0 happens when an individual does NOT have a "next date" - these should be turned to Na
def fix_dates(data,date_var):
    data['new_col'] = data[date_var].astype(str).str[0:4].astype(int)
    data.loc[data['new_col']>2261, date_var] = '2261-01-02'
    data[date_var] = data[date_var].replace(0,np.nan)
    data.loc[data[date_var]=="0", date_var] = None
    data[date_var] = pd.to_datetime(data[date_var],format='%Y-%m-%d',errors='coerce')
    #df[date_var] = pd.to_datetime(df[date_var].str.split(n=1).str[0],format='%Y-%m-%d')
    return data


def get_recidivism_label(data,num_years=1):
    data['Time_Diff'] = pd.DatetimeIndex(data['NextStart']).year - pd.DatetimeIndex(data['END_DATE']).year
    data['Recidivate'] = np.nan
    # if NextPrefix != 0:
    data.loc[(data['NextPrefix']!=0) & (data['Time_Diff']<= num_years) & (data['Time_Diff']>=0), 'Recidivate'] = 1
    data.loc[(data['NextPrefix']!=0) & (data['Time_Diff']> num_years), 'Recidivate'] = 0
    # dealing with small amount of negative Time_diff - data errors or concurrent sentences
    data.loc[(data['NextPrefix']!=0) & (data['Time_Diff']< 0), 'Recidivate'] = 0
    
    
    # if nextprefix = 0, inmate is inactive, and they did not die in prison 
    # (e.g. serving life sentence or  other wise) then 
    # recidivism = 0
    data.loc[(data['NextPrefix']==0) & (data['INMATE_ADMIN_STATUS_CODE']=='INACTIVE') & (data['TYPE_OF_LAST_INMATE_MOVEMENT']!='DEATH'), 'Recidivate'] = 0
    
    # if nextprefix = 0, inmate status code is not active or inactive(could be missing) and 
    # end date is not 2261-01-02 (life sentence), they were likely released from prison
    # recidivism = 0
    data.loc[(data['NextPrefix']==0) & (data['INMATE_ADMIN_STATUS_CODE']!='ACTIVE') & (data['INMATE_ADMIN_STATUS_CODE']!='INACTIVE') & (data['END_DATE']!='2261-01-02'), 'Recidivate'] = 0
    
    return data

In [227]:
# Part D continued
# Step 3. 
# call fix dates function to fix relevant dates
print("Fix Dates")
dataset_flag = fix_dates(dataset_flag,'EARLIEST_SENTENCE_EFFECTIVE_DT')
dataset_flag = fix_dates(dataset_flag,'END_DATE')
dataset_flag = fix_dates(dataset_flag,'NextStart')

# Step 4
# get recidivism flag - see decision rules and function above 
print("Get recidivism flag")
dataset_flag = get_recidivism_label(dataset_flag)


Fix Dates
Get recidivism flag


In [228]:
# Part E - querying additional features
conn = create_connection(config.database_name)
dataset_flag.to_sql('dataset_AB', conn,if_exists='replace', index=False)

query = '''
        SELECT INMATE_DOC_NUMBER as ID,
                DISCIPLINARY_INFRACTION_DATE,
                COMMITMENT_PREFIX,
                EARLIEST_SENTENCE_EFFECTIVE_DT,
                END_DATE,
                COUNT(DISCIPLINARY_INFRACTION_DATE) as INFRACTION_PER_SENT
        FROM INMT9CF1 A
        INNER JOIN dataset_AB B
        WHERE A.INMATE_DOC_NUMBER = B.ID
        AND A.DISCIPLINARY_INFRACTION_DATE >= B.EARLIEST_SENTENCE_EFFECTIVE_DT
        AND A.DISCIPLINARY_INFRACTION_DATE <= B.END_DATE
        GROUP BY INMATE_DOC_NUMBER, COMMITMENT_PREFIX
        ;
        
        '''

disc_infraction = qd.query_db_notebook(conn,query)
conn.close


<function Connection.close>

In [229]:
# Divide infractions by # of sentences if there are dups on ID / DISCIPLINARY_INFRACTION_DATE
    # might indicate concurrent sentences
count_dups = disc_infraction.groupby(['ID','DISCIPLINARY_INFRACTION_DATE'])["ID"].count().reset_index(name="count")
disc_infraction = disc_infraction.merge(count_dups, how = 'left')
disc_infraction['INFRACTION_PER_SENT'] = round(disc_infraction['INFRACTION_PER_SENT']/disc_infraction['count'])

disc_infraction = disc_infraction.loc[:,['ID','COMMITMENT_PREFIX','INFRACTION_PER_SENT']]
print("Disc Infractions",disc_infraction.shape)

# Merge on disciplinary infractions, replace missing to 0
dataset_flag = dataset_flag.merge(disc_infraction, how='left', on=['ID','COMMITMENT_PREFIX'])
dataset_flag.loc[dataset_flag['INFRACTION_PER_SENT'].isnull(),'INFRACTION_PER_SENT'] = 0


Disc Infractions (362962, 3)


In [231]:
# Part E - calculating additional features - getting total # of felony charges and total # of misd charges from
# sentence component
sent_count_fel_misd = sent_comp_small.groupby(['ID','COMMITMENT_PREFIX','PRIMARY_FELONYMISDEMEANOR_CD']).size().reset_index(name='Count')
sent_count_fel_misd = sent_count_fel_misd.set_index(['ID','COMMITMENT_PREFIX','PRIMARY_FELONYMISDEMEANOR_CD']).unstack().reset_index()
sent_count_fel_misd.fillna(0, inplace=True)



In [232]:
sent_count_fel_misd.head()

ID COMMITMENT_PREFIX Count      
PRIMARY_FELONYMISDEMEANOR_CD                            FELON MISD.
0                             0000004                AA   2.0   0.0
1                             0000006                AA   0.0   1.0
2                             0000006                AB   0.0  27.0
3                             0000008                AA   0.0   1.0
4                             0000008                AB   1.0   0.0

In [233]:
dataset_flag = dataset_flag.merge(sent_count_fel_misd, how='left', on =['ID','COMMITMENT_PREFIX'], right_index=False )

/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/daminisharma/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [234]:
#sent_count_fel_misd.set_index(['ID','COMMITMENT_PREFIX'])

In [235]:
#small = sent_count_fel_misd.sample(50)

In [236]:
#dataset_flag.head()

In [237]:
#small.reset_index(level='')

In [238]:
#small = small.loc[:,['ID','COMMITMENT_PREFIX','count']]

In [239]:
#small.columns

In [240]:
#small.reset_index().drop('PRIMARY_FELONYMISDEMEANOR_CD',axis=1)

In [241]:
#sent_count_fel_misd

In [242]:
#index = pd.MultiIndex(sent_count_fel_misd, names=['ID', 'COMMITMENT_PREFIX'])

In [243]:
#sent_count_fel_misd2 = sent_count_fel_misd.pivot(index=index,columns='PRIMARY_FELONYMISDEMEANOR_CD',values='count')



In [244]:
#dataset_flag = dataset_flag.merge(sent_count_fel_misd, how='left', on=['ID','COMMITMENT_PREFIX'])
dataset_flag.isnull().sum()

ID                                     0
COMMITMENT_PREFIX                      0
EARLIEST_SENTENCE_EFFECTIVE_DT         0
MOST_SERIOUS_OFFENSE_CODE              0
INMATE_COMPUTATION_STATUS_FLAG         0
END_DATE                               0
PROJ_END_DATE                          0
INMATE_RECORD_STATUS_CODE           5930
INMATE_ADMIN_STATUS_CODE            5930
DATE_OF_LAST_INMATE_MOVEMENT        5930
TYPE_OF_LAST_INMATE_MOVEMENT       20003
CURRENT_COMMITMENT_PREFIX         190326
CONTROL_STATUS                      5930
GENDER                                 0
RACE                                   2
BIRTH_DATE                             0
STATE_BORN                         30883
ETHNICITY                            704
CITIZENSHIP                          681
PRIMARY_OFFENSE_CODE               44127
NextPrefix                             0
NextStart                         461384
NextOffense                            0
new_col                                0
Time_Diff       

In [245]:
# Part F
# Step 5
# Hold out active senteces
print("Hold out active sentences")
active_sentences = dataset_flag[(dataset_flag['INMATE_ADMIN_STATUS_CODE']=='ACTIVE') & (dataset_flag['NextPrefix']==0)]
print("Size of active sentences dataset: ",active_sentences.shape[0])

# Step 6
# drop observations with no recidivism flag (this will also drop active sentences, but we've already separated those)
print("Drop observations with no recidivism flag (this will also drop active sentences, but we've already separated those)")
print("Additional observations dropped are mostly of those who died in prison and therefore wont have a recidivate flag")
dataset_flag = dataset_flag[(dataset_flag['Recidivate'].notnull())]
print("Size of remaining dataset: ",dataset_flag.shape[0])

# Step 7
# Bring in coded offenses - sanity check

# this merges our coded offenses onto "most serious offense" to check how much coverage
# our variable is giving us. however, this not what we ultimately want - in the end, we want
# our codes to be merged onto "nextOffense" - i.e., the offense code for the next offense 
# someone committed that resulted in re-incarceration
# NextOffense can be missing for 2 reasons: because most serious offense is missing, or because
# the individual did not recidivate. after merging our codes onto "NextOffense", we can replace
# "Decided Category" with 0 if recidivism = 0, and leave it as NA otherwise
#dataset_with_offenses_test = dataset_flag.merge(coded_offenses, how='left', left_on='MOST_SERIOUS_OFFENSE_CODE', right_on='Primary offense code')

# Step 8 and 9
# Now, merge on coded offenses onto NextOffense, turn Decided Category, More Lenient, and more harsh = 0 if recidivism = 0
print("Merging on our coded categories")
dataset_with_offenses = dataset_flag.merge(coded_offenses, how='left', left_on='NextOffense', right_on='Primary offense code')
dataset_with_offenses.loc[dataset_with_offenses['Recidivate']==0,'Decided Category'] = 0
dataset_with_offenses.loc[dataset_with_offenses['Recidivate']==0,'More lenient'] = 0
dataset_with_offenses.loc[dataset_with_offenses['Recidivate']==0,'More harsh'] = 0

print("% missing decided category",dataset_with_offenses['Decided Category'].isnull().sum()/dataset_with_offenses.shape[0])

# Drop those missing decided category
dataset_with_offenses = dataset_with_offenses[(dataset_with_offenses['Decided Category'].notnull())]
print("Dataset size: " , dataset_with_offenses.shape[0])


# Step 10 
# Add active sentences back in so we can merge our coded categories onto Most Serious Offense and so 
# all the data is together when we construct features we'll need before pre processing (e.g. economic vars,
# age vars)
dataset_with_offenses = dataset_with_offenses.append(active_sentences)
# Rename Columns
dataset_with_offenses = dataset_with_offenses.rename(columns ={'Decided Category':'Recidivate_Risk_Level'})
dataset_with_offenses = dataset_with_offenses.rename(columns ={'More lenient':'Recidivate_Risk_Level_Lenient'})
dataset_with_offenses = dataset_with_offenses.rename(columns ={'More harsh':'Recidivate_Risk_Level_Harsh'})

dataset_with_offenses = dataset_with_offenses.merge(coded_offenses, how='left', left_on='MOST_SERIOUS_OFFENSE_CODE', right_on='Primary offense code')
dataset_with_offenses = dataset_with_offenses.rename(columns = {'Decided Category':'Current_Offense_Risk_Level','More lenient':'Current_Offense_Risk_Level_Lenient','More harsh':'Current_Offense_Risk_Level_Harsh'})



Hold out active sentences
Size of active sentences dataset:  32801
Drop observations with no recidivism flag (this will also drop active sentences, but we've already separated those)
Additional observations dropped are mostly of those who died in prison and therefore wont have a recidivate flag
Size of remaining dataset:  827731
Merging on our coded categories
% missing decided category 0.007752518632260964
Dataset size:  821314


In [145]:
#dataset_with_offenses = dataset_with_offenses.append(active_sentences)

In [248]:
dataset_with_offenses.describe()

,new_col,Time_Diff,Recidivate,INFRACTION_PER_SENT,"(Count, FELON)","(Count, MISD.)",Recidivate_Risk_Level,Recidivate_Risk_Level_Lenient,Recidivate_Risk_Level_Harsh,Current_Offense_Risk_Level,Current_Offense_Risk_Level_Lenient,Current_Offense_Risk_Level_Harsh
count,854115.000000,397016.000000,821314.000000,854115.000000,854072.000000,854072.000000,821314.000000,821314.000000,821314.000000,807864.000000,807864.000000,807864.000000
mean,930.329291,3.624128,0.171411,3.898150,1.058833,0.843916,0.390327,0.323152,0.467961,2.351235,1.994487,2.758052
std,998.275713,4.158104,0.376868,14.386806,1.242654,1.528526,0.961667,0.814176,1.145918,1.087621,1.002935,1.225553
min,0.000000,-265.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000
50%,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,3.000000
75%,2002.000000,5.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,3.000000,2.000000,4.000000
max,2020.000000,45.000000,1.000000,1009.000000,87.000000,125.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [249]:
dataset_with_offenses.to_csv('../data/dataset_main_active.csv', index=False)
#active_sentences.to_csv('../data/active_sentences.csv', index=False)

In [247]:
dataset_with_offenses.head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,PRIMARY_OFFENSE_CODE,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,INFRACTION_PER_SENT,"(Count, FELON)","(Count, MISD.)",Primary offense code_x,Description (if needed)_x,Recidivate_Risk_Level,Needed a check?_x,Recidivate_Risk_Level_Lenient,Recidivate_Risk_Level_Harsh,Primary offense code_y,Description (if needed)_y,Current_Offense_Risk_Level,Needed a check?_y,Current_Offense_Risk_Level_Lenient,Current_Offense_Risk_Level_Harsh
0,0000004,AA,1983-07-12,SELL SCHEDULE II,EXPIRED,1984-07-11,1984-07-11,INACTIVE,INACTIVE,1984-07-11,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1961-10-15,ALASKA,UNKNOWN,BORN IN U.S.,SELL SCHEDULE II,0,NaT,0,0,NaN,0.0,0.0,2.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,SELL SCHEDULE II,Selling drugs on Schedule II - high potential ...,3.0,YES,2.0,4.0
1,0000006,AA,1973-01-30,WORTHLESS CHECK,EXPIRED,1973-03-28,0001-01-01,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,WORTHLESS CHECK,AB,1973-04-11,WORTHLESS CHECK,1973,0.0,1.0,0.0,0.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0
2,0000006,AB,1973-04-11,WORTHLESS CHECK,EXPIRED,1975-08-18,1974-08-10,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,WORTHLESS CHECK,0,NaT,0,0,NaN,0.0,0.0,0.0,27.0,NaN,NaN,0.0,NaN,0.0,0.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0
3,0000008,AA,1990-04-09,DWI DRIVING WHILE IMPAIRED,EXPIRED,1990-05-17,1990-10-09,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,DWI DRIVING WHILE IMPAIRED,AB,1993-08-30,HABITUAL IMPAIRED DRIVING,1993,3.0,0.0,0.0,0.0,1.0,HABITUAL IMPAIRED DRIVING,0,0.0,NO,0.0,0.0,DWI DRIVING WHILE IMPAIRED,0,2.0,NO,2.0,2.0
4,0000008,AB,1993-08-30,HABITUAL IMPAIRED DRIVING,EXPIRED,1994-01-26,1994-02-18,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,None,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,HABITUAL IMPAIRED DRIVING,BA,1995-01-02,HABITUAL IMPAIRED DRIVING,1995,1.0,1.0,0.0,1.0,0.0,HABITUAL IMPAIRED DRIVING,0,3.0,NO,3.0,3.0,HABITUAL IMPAIRED DRIVING,0,3.0,NO,3.0,3.0


In [246]:
#active_sentences.head()

In [139]:
#count = active_sentences.groupby("MOST_SERIOUS_OFFENSE_CODE").size().reset_index(name="count")


In [140]:
#count = active_sentences.groupby('MOST_SERIOUS_OFFENSE_CODE')['ID'].count()

In [143]:
#count.sort_values(by='count')

### As detailed above, here's where we stand with "most serious offense code"
    - 33% of dataset A is missing most_serious_offense
    - using sentence component, we created primary offense code for about 92% of the sentence component data (using minimum and maximum length) 
    - this variable (call it Offense_Constructed) has a 93% match rate with MosT Serious Offense in dataset A (where its available)
    - we're going to use Most Serious Offense where available (66% of the time), replace with Offense_Constructed where Most Serious Offense is unavailable and Offense_Constructed is available (32% of data). 
    - This will mean we are still missing Most Serious Offense for 4% of observations. Not all of these will be relevant to our outcome variable (only relevant when someone recidivates) but a) we want to use most serious offense as a predictor so missingness is relevant and b) how many of these are relevant for recidivating might keep changing depending on our # of years for recidivating. After we do all other changes to this dataset (e.g. dropping for weird dates) will check again how many are missing most serious offense. will drop those at that point (2.6% obs)
    - Finally, ~1% of the remaining data is missing our outcome variable once it is merged on because we only coded
       up offenses that took up 95% of all offenses. We drop these as well

In [442]:
# Part C broken down in more detail
    # 1. Deal with date issues (takes us from 903,000 obs to 888,120). For more details, see below:
        # a. replace end date with projected end date where END_DATE = 0001-01-01 (placeholder for missing)
        # b. drop observations still missing end_date (should be only about ~350 observations)
        # c. drop observations missing EARLIEST_SENTENCE_EFFECTIVE_DT (about 12k observations)
        # b and c are dropping those where the sentence is either only in court commitment or only in sentence comp
    # 2. Query the remaining dataset to get the the next commitment prefix, next sentence date, and most serious
        # offense code for the next observation - where all of these exist. for a sentence that does not result in 
        # recidivism, nextprefix, nextstart, nextoffense will be 0
    # 3. Clean up dates - turn them into date format, after recoding the top coded 9999 dates (for life sentences)
    # 4. Get recidivism flags. See decision rule below
    # 5. Hold out active sentences (~approx 32,000 obs)
    # 6. Drop observation with no recidivism flag (Takes us from 888,120 to 850970, i.e. dropping
        # 38,000 observations. 32k of those are active sentences, 6k are "out of universe" i.e.
        # sentences that are expired but the individual was never released (mostly death in prison)
    # 7. Sanity check - Merge on our coded offenses to most serious offenses and see how well we cover the offenses
        # Approx 5% of observations that have Most Serious Offense do NOT have "Decided Category" (our variable)
        # This makes sense because we only coded up offenses that made up 95% of the offenses
    # 8. Merge on our coded offenses to "NextOffense" - the relevant variable now is "Decided category".
    # 9. Replace Decided Category to 0 if recidivism = 0 ; leave it as NA otherwise
        # After holding out active sentences and dropping "out of universe observations", we have ~850k observations
        # of these, we are missing a "Decided Category" flag (as defined by our coded offenses) for 7% of the data
        # this is a lot better than missing it for 33% of the data (since we're missing "most serious offense" for 
        # 33% of the data) but its still not great - hopefully once we bring in most serious offense from sentence
        # component, we can reduce 7% down to something more negligible
    # 10. Understand the missingness of our possible features
    
# We now have two datasets that are ready for pre processing and feature engineering:
    # dataset_with_offenses = datasetA 
    # active_sentences = data on which we will apply our predictions
        # Next steps (I think): develop a list of features and functions that can clean up those features, which can 
        # be applied to both of the datasets above
        # Additionally - do we want to write both of these to csv that we push to github?
    
# More details on Dates
    # In addition to the dates that are null (see above) because some data exists in court commitment
    # that doesnt exist in sentence computation (and vice versa) we also have start and end
    # dates that are 0001-01-01 - based on looking up some offenders with these dates, these
    # are often just missing so 0001-01-01 is a placeholder for missing date

    # There are about 10k observations with end_date = 0001-01-01. These don't seem random -
    # 9k of these are for the commitment prefix BA, and on spot checking many of them look like
    # the sentences were categorized as "FAIR FELONS" - related to the fair sentencing act that
    # affects sentences from 1982 to October 1994 (before NC enacted structured sentencing which
    # abolished parole). It also seems like many of those sentences are missing an "actual release
    # date" from prison but have a release date from parole
    # 
    # Where available, the end date will be replaced with the projected release date. on spot
    # checking, this seems to be a reasonable proxy for when inmate was last moved
    # There are 397 observations missing both end date and projected end date - dropping these
    #
    # About 12k observations have start date = 0001-01-01. On spot checking, some of these
    # appear to be entirely missing from sentence component and from the offender's online
    # profile - as if the sentences were removed ex-post. Since there is no way to get a start
    # date for these, they will be dropped. Approx 1% of the data

# Note on "Active" Flag    
    # To get "Active" sentences, we should probably not trust the Inmate Commitment
    # status flag in court commitment. This often appears active even for sentences that
    # online show "service status" = "Expired"

    # instead, we should merge on information from INmate Profile. This has "inmate record status"
    # and "inmate admin status". After some exploration, it seems like admin status = active
    # means one is in prison; record status = active (if admin status = inactive) is mostly for
    # people on parole/probation.


# Decision rule for recidivism flag:
    # if NextPrefix != 0: if nextStart - endDate is less than XXX (make this a parameter) then recidivism = 1 else 0

    # if nextprefix = 0, inmate is inactive, and they did not die in prison 
    # (e.g. serving life sentence or  other wise) then 
    # recidivism = 0

    # if nextprefix = 0, inmate status code is not active or inactive (could be missing) and 
    # end date is not 2261-01-02 (life sentence), they were likely released from prison
    # recidivism = 0